# Getting the dataset  ready

In [81]:
import pandas as pd
my_csv = pd.read_csv(r"D:\Programming shit\Python\MachineLearningDatasets\creditcard.csv")

In [82]:
import numpy as np
my_csv['split'] = np.random.randn(my_csv.shape[0], 1)

In [83]:
selection = np.random.rand(len(my_csv)) <= 0.85

In [84]:
train_csv = my_csv[selection]
test_csv = my_csv[~selection]

In [85]:
trainreader = csv.reader(train_csv.drop(columns = ['Class','split']), delimiter='t')
testreader = csv.reader(test_csv.drop(columns = ['Class','split']), delimiter='t')
transaction_data_train = []
transaction_data_test = []
for i, line in enumerate(trainreader):
    transaction_data_train.append(line)
for i, line in enumerate(testreader): 
    transaction_data_test.append(line)

In [86]:
transaction_data_train = train_csv.drop(columns = ['Class','split'])
transaction_data_test = test_csv.drop(columns = ['Class','split'])

In [87]:
transaction_labels_train = list(train_csv['Class'])
transaction_labels_test = list(test_csv['Class'])

In [88]:
import torch

class CardTransactionsDataset(torch.utils.data.Dataset):
    def __init__(self,labels,transactions_csv):
        self.labels = labels
        self.data = transactions_csv
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        return self.data[idx],self.labels[idx]
        

In [93]:
from torchvision.transforms.functional import to_tensor

def my_collate(data):
    processed_data = []
    for line in data:
        line = to_tensor(line)
        line = line.unsqueeze(0)
        processed_data.append(line)
        newdata = torch.cat(processed_data,dim = 0)
        return newdata

In [94]:
training_data = CardTransactionsDataset(transaction_labels_train,transaction_data_train)
testing_data = CardTransactionsDataset(transaction_labels_test,transaction_data_test)

In [95]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=1000, shuffle=True,collate_fn=my_collate)
test_dataloader = DataLoader(testing_data, batch_size=1000, shuffle=True,collate_fn=my_collate)